<a href="https://colab.research.google.com/github/vokiet2005/TTNT/blob/main/baituan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 PHẦN 1 — Import thư viện & Khai báo trạng thái

Giải thích (Markdown):

import copy: dùng để sao chép sâu (deep copy) bàn cờ khi tạo trạng thái mới, tránh thay đổi trạng thái gốc.

import math: dùng để lấy math.inf và -math.inf trong Minimax.

import random: (có thể dùng cho AI chọn nước đi ngẫu nhiên nếu cần).

import numpy: dùng để in bàn cờ đẹp dạng ma trận.

Khai báo các trạng thái ô: X (người chơi X), O (người chơi O), EMPTY (ô trống).

Biến user và ai để xác định lượt đi của người dùng và AI.

import copy
import math
import random
import numpy

X = "X"
O = "O"
EMPTY = None

user = None
ai = None

PHẦN 2 — Hàm initial_state()

Giải thích:

Tạo bàn cờ 3×3 trống.

Trả về danh sách 2 chiều với tất cả ô bằng EMPTY.

def initial_state():
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]]

 PHẦN 3 — Hàm player(board)

Giải thích:

Đếm số ô đã đánh (count).

Nếu số ô chẵn → lượt người dùng.

Nếu số ô lẻ → lượt AI.

Giả sử người dùng luôn đi trước.

def player(board):
    count = 0
    for row in board:
        for cell in row:
            if cell is not EMPTY:
                count += 1
    return ai if count % 2 != 0 else user

 PHẦN 4 — Hàm actions(board)

Giải thích:

Trả về tập hợp các hành động hợp lệ (i, j) của ô trống.

Sử dụng set() để không trùng lặp.

def actions(board):
    res = set()
    board_len = len(board)
    for i in range(board_len):
        for j in range(board_len):
            if board[i][j] == EMPTY:
                res.add((i, j))
    return res

PHẦN 5 — Hàm result(board, action)

Giải thích:

Tạo bản sao bàn cờ (deepcopy).

Xác định lượt chơi hiện tại.

Thực hiện nước đi (i, j) cho người chơi hiện tại.

Trả về trạng thái bàn cờ mới.

def result(board, action):
    curr_player = player(board)
    result_board = copy.deepcopy(board)
    (i, j) = action
    result_board[i][j] = curr_player
    return result_board

 PHẦN 6 — Hàm kiểm tra người thắng
6.1. Kiểm tra theo hàng ngang
def get_horizontal_winner(board):
    winner_val = None
    board_len = len(board)
    for i in range(board_len):
        winner_val = board[i][0]
        if winner_val is EMPTY:
            continue
        for j in range(board_len):
            if board[i][j] != winner_val:
                winner_val = None
                break
        if winner_val:
            return winner_val
    return winner_val

6.2. Kiểm tra theo cột dọc
def get_vertical_winner(board):
    winner_val = None
    board_len = len(board)
    for i in range(board_len):
        winner_val = board[0][i]
        if winner_val is EMPTY:
            continue
        for j in range(board_len):
            if board[j][i] != winner_val:
                winner_val = None
                break
        if winner_val:
            return winner_val
    return winner_val

6.3. Kiểm tra theo đường chéo
def get_diagonal_winner(board):
    winner_val = board[0][0]
    board_len = len(board)
    if winner_val is not EMPTY:
        if all(board[i][i] == winner_val for i in range(board_len)):
            return winner_val

    winner_val = board[0][board_len - 1]
    if winner_val is not EMPTY:
        if all(board[i][board_len - 1 - i] == winner_val for i in range(board_len)):
            return winner_val

    return None

 PHẦN 7 — Hàm winner(board)

Giải thích:

Kết hợp kiểm tra hàng ngang, cột dọc, chéo.

Trả về người thắng nếu có, hoặc None.

def winner(board):
    return get_horizontal_winner(board) or \
           get_vertical_winner(board) or \
           get_diagonal_winner(board)

 PHẦN 8 — Hàm terminal(board)

Giải thích:

Trò chơi kết thúc nếu:

Có người thắng (winner(board) != None)

Hết ô trống → hòa

def terminal(board):
    if winner(board) is not None:
        return True
    for row in board:
        for cell in row:
            if cell == EMPTY:
                return False
    return True

 PHẦN 9 — Hàm utility(board)

Giải thích:

X thắng → trả về 1

O thắng → trả về -1

Hòa → 0

def utility(board):
    winner_val = winner(board)
    if winner_val == X:
        return 1
    elif winner_val == O:
        return -1
    return 0

 PHẦN 10 — Minimax (maxValue & minValue)

Giải thích:

maxValue: tìm giá trị tối đa cho X

minValue: tìm giá trị tối thiểu cho O

Đệ quy duyệt tất cả nước đi

def maxValue(state):
    if terminal(state):
        return utility(state)
    v = -math.inf
    for action in actions(state):
        v = max(v, minValue(result(state, action)))
    return v

def minValue(state):
    if terminal(state):
        return utility(state)
    v = math.inf
    for action in actions(state):
        v = min(v, maxValue(result(state, action)))
    return v

 PHẦN 11 — Hàm minimax(board)

Giải thích:

Chọn nước đi tối ưu cho người chơi hiện tại.

Nếu là X → tìm maxValue

Nếu là O → tìm minValue

def minimax(board):
    current_player = player(board)
    if current_player == X:
        max_val = -math.inf
        move = None
        for action in actions(board):
            check = minValue(result(board, action))
            if check > max_val:
                max_val = check
                move = action
        return move
    else:
        min_val = math.inf
        move = None
        for action in actions(board):
            check = maxValue(result(board, action))
            if check < min_val:
                min_val = check
                move = action
        return move

 PHẦN 12 — Vòng lặp chính (Gameplay)

Giải thích:

Chọn người chơi (X hoặc O).

In bàn cờ hiện tại trước và sau mỗi nước đi.

Lượt AI → gọi minimax()

Lượt người dùng → nhập hàng và cột

Kiểm tra hợp lệ.

Kết thúc nếu thắng hoặc hòa.

if __name__ == "__main__":
    board = initial_state()
    print("Chọn người chơi (X hoặc O):")
    user_input = input().upper()
    while user_input not in ["X", "O"]:
        print("Lựa chọn không hợp lệ. Chọn X hoặc O.")
        user_input = input().upper()
    user = user_input
    ai = "O" if user == "X" else "X"
    print(f"Bạn là {user}. AI là {ai}.")

    while True:
        if terminal(board):
            winner_val = winner(board)
            if winner_val is None:
                print("Hòa!")
            else:
                print(f"{winner_val} thắng!")
            break

        if player(board) == ai:
            print(f"AI ({ai}) đang đi...")
            move = minimax(board)
            board = result(board, move)
            print("Bàn cờ sau nước đi AI:")
            print(numpy.array(board))
        else:
            print(f"Lượt của bạn ({user}):")
            print(numpy.array(board))
            valid_move = False
            while not valid_move:
                try:
                    i = int(input("Hàng (0-2): "))
                    j = int(input("Cột (0-2): "))
                    if 0 <= i < 3 and 0 <= j < 3 and board[i][j] == EMPTY:
                        board = result(board, (i, j))
                        valid_move = True
                    else:
                        print("Ô không hợp lệ. Chọn ô trống trong 0-2.")
                except ValueError:
                    print("Nhập số hợp lệ.")

In [ ]:
import copy
import math
import random
import numpy

# Định nghĩa các trạng thái có thể của một ô trên bàn cờ Tic-Tac-Toe
X = "X"  # Người chơi X
O = "O"  # Người chơi O
EMPTY = None  # Ô trống

# Khởi tạo các biến người chơi (sẽ được thiết lập bằng đầu vào người dùng sau)
user = None
ai = None

def initial_state():
    """
    Trả về trạng thái bắt đầu của bàn cờ.
    Bàn cờ là một lưới 3x3, ban đầu tất cả các ô đều EMPTY.
    """
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]]

def player(board):
    """
    Trả về người chơi có lượt đi tiếp theo trên bàn cờ.
    Đếm số ô không trống. Nếu số ô lẻ, đó là lượt đi của AI (vì người dùng bắt đầu).
    Nếu số ô chẵn, đó là lượt đi của người dùng.
    """
    count = 0
    for row in board:
        for cell in row:
            if cell is not EMPTY:  # Nếu ô không trống, một nước đi đã được thực hiện
                count += 1

    # Nếu số nước đi lẻ, đó là lượt của AI.
    # Nếu số nước đi chẵn, đó là lượt của người dùng.
    # Giả sử 'user' luôn bắt đầu trò chơi.
    if count % 2 != 0: # Sau nước đi đầu tiên của người dùng, count là 1, AI chơi. Sau nước đi của AI, count là 2, người dùng chơi.
        return ai
    return user

def actions(board):
    """
    Trả về tập hợp tất cả các hành động có thể (i, j) có sẵn trên bàn cờ.
    Một hành động được biểu thị bằng một tuple (hàng, cột) của một ô trống.
    """
    res = set()
    board_len = len(board) # Đối với bàn cờ 3x3, board_len là 3
    for i in range(board_len):
        for j in range(board_len):
            if board[i][j] == EMPTY: # Nếu ô trống, đó là một nước đi có thể
                res.add((i, j))
    return res

def result(board, action):
    """
    Trả về bàn cờ là kết quả của việc thực hiện nước đi (i, j) trên bàn cờ.
    Hàm này không sửa đổi bàn cờ gốc; nó trả về một trạng thái bàn cờ mới.
    """
    curr_player = player(board) # Xác định lượt của ai
    result_board = copy.deepcopy(board) # Tạo một bản sao sâu để tránh sửa đổi bàn cờ gốc
    (i, j) = action # Giải nén hành động (hàng, cột)
    result_board[i][j] = curr_player # Đặt dấu của người chơi hiện tại lên bàn cờ
    return result_board

def get_horizontal_winner(board):
    """
    Kiểm tra người thắng theo chiều ngang.
    """
    winner_val = None
    board_len = len(board)
    for i in range(board_len): # Lặp qua từng hàng
        winner_val = board[i][0] # Giả sử ô đầu tiên trong hàng là dấu của người thắng
        if winner_val is EMPTY: # Nếu ô đầu tiên trống, chưa có người thắng nào trong hàng này
            continue
        for j in range(board_len): # Kiểm tra tất cả các ô trong hàng hiện tại
            if board[i][j] != winner_val: # Nếu bất kỳ ô nào không khớp với dấu của người thắng được giả định
                winner_val = None # Không có người thắng trong hàng này
                break
        if winner_val: # Nếu tìm thấy người thắng trong hàng này
            return winner_val
    return winner_val # Không tìm thấy người thắng theo chiều ngang

def get_vertical_winner(board):
    """
    Kiểm tra người thắng theo chiều dọc.
    """
    winner_val = None
    board_len = len(board)
    for i in range(board_len): # Lặp qua từng chỉ số cột
        winner_val = board[0][i] # Giả sử ô đầu tiên trong cột là dấu của người thắng
        if winner_val is EMPTY: # Nếu ô đầu tiên trống, chưa có người thắng nào trong cột này
            continue
        for j in range(board_len): # Kiểm tra tất cả các ô trong cột hiện tại
            if board[j][i] != winner_val: # Nếu bất kỳ ô nào không khớp với dấu của người thắng được giả định
                winner_val = None # Không có người thắng trong cột này
                break
        if winner_val: # Nếu tìm thấy người thắng trong cột này
            return winner_val
    return winner_val # Không tìm thấy người thắng theo chiều dọc

def get_diagonal_winner(board):
    """
    Kiểm tra người thắng theo đường chéo (cả đường chéo chính và đường chéo phụ).
    """
    winner_val = None
    board_len = len(board)

    # Kiểm tra đường chéo chính (trên cùng bên trái đến dưới cùng bên phải)
    winner_val = board[0][0] # Giả sử ô đầu tiên là dấu của người thắng
    if winner_val is not EMPTY: # Chỉ kiểm tra nếu ô đầu tiên không trống
        found_winner = True
        for i in range(board_len):
            if board[i][i] != winner_val: # Nếu bất kỳ ô nào không khớp
                found_winner = False
                break
        if found_winner: # Nếu tất cả các ô khớp
            return winner_val

    # Kiểm tra đường chéo phụ (trên cùng bên phải đến dưới cùng bên trái)
    winner_val = board[0][board_len - 1] # Giả sử ô đầu tiên là dấu của người thắng
    if winner_val is not EMPTY: # Chỉ kiểm tra nếu ô đầu tiên không trống
        found_winner = True
        for i in range(board_len):
            j = board_len - 1 - i # Tính chỉ số cột cho đường chéo phụ
            if board[i][j] != winner_val: # Nếu bất kỳ ô nào không khớp
                found_winner = False
                break
        if found_winner: # Nếu tất cả các ô khớp
            return winner_val

    return None # Không tìm thấy người thắng theo đường chéo

def winner(board):
    """
    Trả về người thắng cuộc, nếu có.
    Kết hợp kiểm tra người thắng theo chiều ngang, dọc và đường chéo.
    """
    # Sử dụng 'or' để trả về người thắng đầu tiên không phải None được tìm thấy
    winner_val = get_horizontal_winner(board) or \
                 get_vertical_winner(board) or \
                 get_diagonal_winner(board)
    return winner_val

def terminal(board):
    """
    Trả về True nếu trò chơi kết thúc (có người thắng hoặc hòa), False nếu ngược lại.
    """
    if winner(board) is not None: # Nếu có người thắng, trò chơi kết thúc
        return True

    # Kiểm tra hòa (không còn ô trống nào)
    for row in board:
        for cell in row:
            if cell == EMPTY: # Nếu bất kỳ ô nào trống, trò chơi chưa kết thúc
                return False
    return True # Không có người thắng và không còn ô trống nào có nghĩa là hòa, trò chơi kết thúc

def utility(board):
    """
    Trả về 1 nếu X thắng, -1 nếu O thắng, 0 nếu hòa.
    Điều này được sử dụng cho thuật toán minimax để đánh giá các trạng thái bàn cờ.
    """
    winner_val = winner(board)
    if winner_val == X:
        return 1  # X thắng
    elif winner_val == O:
        return -1 # O thắng
    return 0  # Hòa hoặc trò chơi chưa kết thúc

def maxValue(state):
    """
    Tìm giá trị tiện ích tối đa cho người chơi tối đa hóa (X).
    Được sử dụng trong thuật toán minimax.
    """
    if terminal(state): # Trường hợp cơ sở: nếu trò chơi kết thúc, trả về tiện ích của nó
        return utility(state)
    v = -math.inf # Khởi tạo v thành âm vô cùng
    for action in actions(state): # Lặp qua tất cả các nước đi có thể
        v = max(v, minValue(result(state, action))) # Gọi đệ quy minValue cho trạng thái tiếp theo
    return v

def minValue(state):
    """
    Tìm giá trị tiện ích tối thiểu cho người chơi tối thiểu hóa (O).
    Được sử dụng trong thuật toán minimax.
    """
    if terminal(state): # Trường hợp cơ sở: nếu trò chơi kết thúc, trả về tiện ích của nó
        return utility(state)
    v = math.inf # Khởi tạo v thành dương vô cùng
    for action in actions(state): # Lặp qua tất cả các nước đi có thể
        v = min(v, maxValue(result(state, action))) # Gọi đệ quy maxValue cho trạng thái tiếp theo
    return v

def minimax(board):
    """
    Trả về hành động tối ưu cho người chơi hiện tại trên bàn cờ bằng cách sử dụng thuật toán Minimax.
    Thuật toán khám phá tất cả các nước đi tương lai có thể để xác định nước đi tốt nhất tiếp theo.
    """
    current_player = player(board)

    if current_player == X:  # Người chơi tối đa hóa (X muốn tối đa hóa điểm số)
        min_val = -math.inf # Lưu trữ tiện ích tối đa được tìm thấy cho X cho đến nay
        move = None # Lưu trữ nước đi tốt nhất cho X
        for action in actions(board): # Xem xét từng hành động có thể
            # Đánh giá kết quả của hành động này giả sử cả hai bên đều chơi tối ưu
            check = minValue(result(board, action)) # Sau khi X chơi, đến lượt O (minValue)
            if check > min_val: # Nếu nước đi này dẫn đến kết quả tốt hơn cho X
                min_val = check # Cập nhật tiện ích tốt nhất
                move = action # Cập nhật nước đi tốt nhất
        return move # Trả về nước đi tối ưu cho X
    else:  # Người chơi tối thiểu hóa (O muốn tối thiểu hóa điểm số của X, thực tế là tối đa hóa điểm số của O)
        max_val = math.inf # Lưu trữ tiện ích tối thiểu được tìm thấy cho O cho đến nay
        move = None # Lưu trữ nước đi tốt nhất cho O
        for action in actions(board): # Xem xét từng hành động có thể
            # Đánh giá kết quả của hành động này giả sử cả hai bên đều chơi tối ưu
            check = maxValue(result(board, action)) # Sau khi O chơi, đến lượt X (maxValue)
            if check < max_val: # Nếu nước đi này dẫn đến kết quả tốt hơn cho O
                max_val = check # Cập nhật tiện ích tốt nhất
                move = action # Cập nhật nước đi tốt nhất
        return move # Trả về nước đi tối ưu cho O

# Vòng lặp chính của trò chơi
if __name__ == "__main__":
    board = initial_state() # Khởi tạo một bàn cờ Tic-Tac-Toe trống
    ai_turn = False # Cờ để theo dõi lượt của ai. Người dùng bắt đầu, vì vậy lượt của AI ban đầu là False.

    print("Chọn người chơi (X hoặc O):")
    user_input = input().upper() # Lấy lựa chọn của người dùng và chuyển đổi thành chữ hoa để nhất quán

    # Xác thực đầu vào cho lựa chọn của người dùng (X hoặc O)
    while user_input not in ["X", "O"]:
        print("Lựa chọn không hợp lệ. Vui lòng chọn 'X' hoặc 'O'.")
        user_input = input().upper()

    user = user_input
    if user == "X":
        ai = "O"
    else:
        ai = "X"

    print(f"Bạn là {user}. AI là {ai}.")

    while True:
        game_over = terminal(board) # Kiểm tra xem trò chơi đã kết thúc chưa
        playr = player(board) # Xác định lượt của ai bây giờ

        if game_over: # Nếu trò chơi kết thúc
            winner_val = winner(board) # Lấy người thắng cuộc (hoặc None nếu hòa)
            if winner_val is None:
                print("Trò chơi kết thúc: Hòa.")
            else:
                print(f"Trò chơi kết thúc: {winner_val} thắng.")
            break # Thoát vòng lặp trò chơi
        else:
            # Nếu là lượt của AI và trò chơi chưa kết thúc
            if playr == ai and not game_over:
                print(f"AI ({ai}) đang thực hiện nước đi...")
                move = minimax(board) # AI tính toán nước đi tối ưu
                board = result(board, move) # Cập nhật bàn cờ với nước đi của AI
                print("Nước đi của AI:")
                print(numpy.array(board)) # Hiển thị bàn cờ sau nước đi của AI

            # Nếu là lượt của người dùng và trò chơi chưa kết thúc
            elif playr == user and not game_over:
                print(f"Lượt của bạn ({user}).")
                print("Bàn cờ hiện tại:")
                print(numpy.array(board)) # Hiển thị bàn cờ hiện tại trước nước đi của người dùng

                valid_move = False
                while not valid_move:
                    try:
                        print("Nhập vị trí để di chuyển (hàng,cột), ví dụ: 0 cho hàng đầu tiên, 1 cho cột thứ hai:")
                        i = int(input("Hàng (0-2): "))
                        j = int(input("Cột (0-2): "))

                        # Xác thực đầu vào người dùng: trong giới hạn bàn cờ và ô trống
                        if 0 <= i < 3 and 0 <= j < 3 and board[i][j] == EMPTY:
                            board = result(board, (i, j)) # Cập nhật bàn cờ với nước đi của người dùng
                            valid_move = True # Nước đi hợp lệ, thoát vòng lặp nhập
                        else:
                            print("Nước đi không hợp lệ. Vui lòng chọn một ô trống trong giới hạn bàn cờ (0-2).")
                    except ValueError:
                        print("Đầu vào không hợp lệ. Vui lòng nhập số cho hàng và cột.")


PHẦN 1 — Import thư viện

Giải thích:

os: xóa console để in bàn cờ mới (cls cho Windows, clear cho Linux/Mac).

math: dùng math.inf và -math.inf trong Minimax.

import os, math

 PHẦN 2 — Hàm GetWinner(board)

Giải thích:

Trả về người thắng trên bàn cờ hiện tại.

Kiểm tra:

Hàng ngang

Cột dọc

Đường chéo

Nếu không ai thắng → trả về None.

def GetWinner(board):
    # horizontal
    if board[0] == board[1] == board[2]:
        return board[0]
    elif board[3] == board[4] == board[5]:
        return board[3]
    elif board[6] == board[7] == board[8]:
        return board[6]
    # vertical
    elif board[0] == board[3] == board[6]:
        return board[0]
    elif board[1] == board[4] == board[7]:
        return board[1]
    elif board[2] == board[5] == board[8]:
        return board[2]
    # diagonal
    elif board[0] == board[4] == board[8]:
        return board[0]
    elif board[2] == board[4] == board[6]:
        return board[2]
    return None

PHẦN 3 — Hàm PrintBoard(board)

Giải thích:

Xóa màn hình console trước khi in bàn cờ mới.

In bàn cờ dạng 3x3.

def PrintBoard(board):
    os.system('cls' if os.name=='nt' else 'clear')
    print(f'''
{board[0]}|{board[1]}|{board[2]}
{board[3]}|{board[4]}|{board[5]}
{board[6]}|{board[7]}|{board[8]}
''')

 PHẦN 4 — Hàm GetAvailableCells(board)

Giải thích:

Trả về danh sách các ô trống.

Nếu ô không phải "X" hay "O" → thêm vào danh sách.

def GetAvailableCells(board):
    available = []
    for cell in board:
        if cell != "X" and cell != "O":
            available.append(cell)
    return available

 PHẦN 5 — Hàm minimax(position, depth, alpha, beta, isMaximizing)

Giải thích:

Thuật toán AI: chọn nước đi tốt nhất.

Parameters:

position: trạng thái bàn cờ hiện tại

depth: độ sâu của đệ quy

alpha, beta: giá trị cắt tỉa Alpha-Beta

isMaximizing: True nếu lượt X (Max), False nếu O (Min)

Logic:

Nếu có người thắng → trả giá trị +10 - depth cho X thắng, -10 + depth cho O thắng.

Nếu hòa → trả về 0

Lặp qua các ô trống → đặt dấu → gọi đệ quy → cắt tỉa Alpha-Beta nếu cần.

def minimax(position, depth, alpha, beta, isMaximizing):
    winner = GetWinner(position)
    if winner != None:
        return 10 - depth if winner == "X" else -10 + depth
    if len(GetAvailableCells(position)) == 0:
        return 0

    if isMaximizing:
        maxEval = -math.inf
        for cell in GetAvailableCells(position):
            position[cell - 1] = "X"
            Eval = minimax(position, depth + 1, alpha, beta, False)
            maxEval = max(maxEval, Eval)
            alpha = max(alpha, Eval)
            position[cell - 1] = cell
            if beta <= alpha:
                break
        return maxEval
    else:
        minEval = +math.inf
        for cell in GetAvailableCells(position):
            position[cell - 1] = "O"
            Eval = minimax(position, depth + 1, alpha, beta, True)
            minEval = min(minEval, Eval)
            beta = min(beta, Eval)
            position[cell - 1] = cell
            if beta <= alpha:
                break
        return minEval

 PHẦN 6 — Hàm FindBestMove(currentPosition, AI)

Giải thích:

Tìm nước đi tối ưu cho AI.

Lặp qua tất cả ô trống → gọi minimax → chọn nước đi Max (X) hoặc Min (O).

def FindBestMove(currentPosition, AI):
    bestVal = -math.inf if AI == "X" else +math.inf
    bestMove = -1
    for cell in GetAvailableCells(currentPosition):
        currentPosition[cell - 1] = AI
        moveVal = minimax(currentPosition, 0, -math.inf, +math.inf,
                           False if AI == "X" else True)
        currentPosition[cell - 1] = cell
        if (AI == "X" and moveVal > bestVal):
            bestMove = cell
            bestVal = moveVal
        elif (AI == "O" and moveVal < bestVal):
            bestMove = cell
            bestVal = moveVal
    return bestMove

PHẦN 7 — Vòng lặp chính (main)

Giải thích:

Người dùng chọn X hoặc O.

AI sẽ nhận ký hiệu còn lại.

Khởi tạo bàn cờ [1,2,...,9].

Lặp tới khi thắng hoặc hòa:

AI → gọi FindBestMove

Người → nhập số ô

In kết quả và bàn cờ cuối.

def main():
    player = input("Play as X or O? ").strip().upper()
    AI = "O" if player == "X" else "X"
    currentGame = [*range(1, 10)]
    currentTurn = "X"
    counter = 0

    while True:
        if currentTurn == AI:
            cell = FindBestMove(currentGame, AI)
            currentGame[cell - 1] = AI
            currentTurn = player
        elif currentTurn == player:
            PrintBoard(currentGame)
            while True:
                humanInput = int(input("Enter Number: ").strip())
                if humanInput in currentGame:
                    currentGame[humanInput - 1] = player
                    currentTurn = AI
                    break
                else:
                    PrintBoard(currentGame)
                    print("Cell Not Available.")
        if GetWinner(currentGame) != None:
            PrintBoard(currentGame)
            print(f"{GetWinner(currentGame)} WON!!!")
            break
        counter += 1
        if GetWinner(currentGame) == None and counter == 9:
            PrintBoard(currentGame)
            print("Tie.")
            break

if __name__ == "__main__":
    main()

In [ ]:
import os, math
def GetWinner(board):
    """
    Returns the winner in the current board if there is one, otherwise it
    returns None.
    """
    # horizontal
    if board[0] == board[1] and board[1] == board[2]:
        return board[0]
    elif board[3] == board[4] and board[4] == board[5]:
        return board[3]
    elif board[6] == board[7] and board[7] == board[8]:
        return board[6]
    # vertical
    elif board[0] == board[3] and board[3] == board[6]:
        return board[0]
    elif board[1] == board[4] and board[4] == board[7]:
        return board[1]
    elif board[2] == board[5] and board[5] == board[8]:
        return board[2]
    # diagonal
    elif board[0] == board[4] and board[4] == board[8]:
        return board[0]
    elif board[2] == board[4] and board[4] == board[6]:
        return board[2]
    return None
def PrintBoard(board):
    """
    Clears the console and prints the current board.
    """
    os.system('cls' if os.name=='nt' else 'clear')
    print(f'''
{board[0]}|{board[1]}|{board[2]}
{board[3]}|{board[4]}|{board[5]}
{board[6]}|{board[7]}|{board[8]}
''')
def GetAvailableCells(board):
    """
    Returns a list of indices containing all available cells in a board.
    """
    available = list()
    for cell in board:
        if cell != "X" and cell != "O":
            available.append(cell)
    return available


def minimax(position, depth, alpha, beta, isMaximizing):
    """
    The AI algorithm responsible for choosing the best move. Returns best
    value of a move.
    """
    # evaluate current board: if maximizing player won -> return 10
    # if minimizing player won -> return -10
    # if no one is winning (tie) -> return 0
    # NOTE: Even though the following AI plays perfectly, it might
    # choose to make a move which will result in a slower victory
    # or a faster loss. Lets take an example and explain it
    # Assume that there are 2 possible ways for X to win the game
    # from a give board state.
    # Move A : X can win in 2 move
    # Move B : X can win in 4 moves
    # Our evaluation will return a value of +10 for both moves A and
    # B. Even though the move A
    # is better because it ensures a faster victory, our AI may
    # choose B sometimes. To overcome
    # this problem we subtract the depth value from the evaluated
    # score. This means that in case
    # of a victory it will choose a the victory which takes least
    # number of moves and in case of
    # a loss it will try to prolong the game and play as many moves
    # as possible.
    winner = GetWinner(position)
    if winner != None:
        return 10 - depth if winner == "X" else -10 + depth
    if len(GetAvailableCells(position)) == 0:
        return 0
    if isMaximizing:
        maxEval = -math.inf
        for cell in GetAvailableCells(position):
            position[cell - 1] = "X"
            Eval = minimax(position, depth + 1, alpha, beta, False)
            maxEval = max(maxEval, Eval)
            alpha = max(alpha, Eval)
            position[cell - 1] = cell
            if beta <= alpha:
                break # prune
        return maxEval
    else:
        minEval = +math.inf
        for cell in GetAvailableCells(position):
            position[cell - 1] = "O"
            Eval = minimax(position, depth + 1, alpha, beta, True)
            minEval = min(minEval, Eval)
            beta = min(beta, Eval)
            position[cell - 1] = cell
            if beta <= alpha:
                break # prune
        return minEval
def FindBestMove(currentPosition, AI):
    """
    This will return the best possible move for the player.
    Will Traverse all cells, evaluate minimax function for all empty cells.
    And return the cell with optimal value.
    Parameters:
    currentPosition (list): The current board to find best move for.
    AI (str): The AI Player ("X" or "O").
    Returns:
    int: Index of best move for the current position.
    """
    bestVal = -math.inf if AI == "X" else +math.inf
    bestMove = -1
    for cell in GetAvailableCells(currentPosition):
        currentPosition[cell - 1] = AI
        moveVal = minimax(currentPosition, 0, -math.inf, +math.inf, False
        if AI == "X" else True)
        currentPosition[cell - 1] = cell
        if (AI == "X" and moveVal > bestVal):
            bestMove = cell
            bestVal = moveVal
        elif (AI == "O" and moveVal < bestVal):
            bestMove = cell
            bestVal = moveVal
    return bestMove
def main():
    player = input("Play as X or O? ").strip().upper()
    AI = "O" if player == "X" else "X"
    currentGame = [*range(1, 10)]
    # X always starts first.
    currentTurn = "X"
    counter = 0
    while True:
        if currentTurn == AI:
            # NOTE: if the AI starts first, it'll always choose index 0 so
            # to save time you could play it.
            cell = FindBestMove(currentGame, AI)
            currentGame[cell - 1] = AI
            currentTurn = player
        elif currentTurn == player:
            PrintBoard(currentGame)
            while True:
                humanInput = int(input("Enter Number: ").strip())
                if humanInput in currentGame:
                    currentGame[humanInput - 1] = player
                    currentTurn = AI
                    break
                else:
                    PrintBoard(currentGame)
                    print("Cell Not Available.")
        if GetWinner(currentGame) != None:
            PrintBoard(currentGame)
            print(f"{GetWinner(currentGame)} WON!!!")
            break
        counter += 1
        if GetWinner(currentGame) == None and counter == 9:
            PrintBoard(currentGame)
            print("Tie.")
            break
if __name__ == "__main__":
    main()

In [ ]:
import math
import copy
import numpy as np

X = "X"
O = "O"
EMPTY = None

# WIN_LENGTH sẽ được người dùng nhập vào hoặc mặc định
WIN_LENGTH = None

# ===== 1. Tạo bàn NxN =====
def initial_state(N):
    return [[EMPTY for _ in range(N)] for _ in range(N)]

# ===== 2. Xác định người chơi tiếp theo =====
def player(board):
    count = sum(1 for row in board for cell in row if cell is not None)
    return X if count % 2 == 0 else O

# ===== 3. Các hành động hợp lệ =====
def actions(board):
    N = len(board)
    return {(i, j) for i in range(N) for j in range(N) if board[i][j] == EMPTY}

# ===== 4. Trả về bảng sau khi đi =====
def result(board, action):
    i, j = action
    new_board = copy.deepcopy(board)
    new_board[i][j] = player(board)
    return new_board

# ===== 5. Kiểm tra thắng K liên tiếp =====
def check_k_in_a_row(board, symbol, k):
    N = len(board)

    if k > N: # Không thể thắng nếu K lớn hơn N
        return False

    # check ngang
    for i in range(N):
        for j in range(N - k + 1):
            if all(board[i][j + l] == symbol for l in range(k)):
                return True

    # check dọc
    for i in range(N - k + 1):
        for j in range(N):
            if all(board[i + l][j] == symbol for l in range(k)):
                return True

    # check chéo chính
    for i in range(N - k + 1):
        for j in range(N - k + 1):
            if all(board[i + l][j + l] == symbol for l in range(k)):
                return True

    # check chéo phụ
    for i in range(N - k + 1):
        for j in range(k - 1, N):
            if all(board[i + l][j - l] == symbol for l in range(k)):
                return True

    return False

# ===== 6. Hàm winner =====
def winner(board):
    # Sử dụng biến WIN_LENGTH toàn cục được thiết lập khi bắt đầu trò chơi
    if check_k_in_a_row(board, X, WIN_LENGTH):
        return X
    if check_k_in_a_row(board, O, WIN_LENGTH):
        return O
    return None

# ===== 7. Kết thúc trận đấu =====
def terminal(board):
    if winner(board) is not None:
        return True
    if not actions(board): # Không còn nước đi nào (hòa)
        return True
    return False

# ===== 8. Utility cho minimax =====
def utility(board):
    w = winner(board)
    if w == X:
        return 1
    if w == O:
        return -1
    return 0

# ===== 9. Minimax với Alpha-Beta =====
def max_value(board, alpha, beta, depth):
    if terminal(board) or depth == 0:
        return utility(board)

    v = -math.inf
    for action in actions(board):
        v = max(v, min_value(result(board, action), alpha, beta, depth - 1))
        if v >= beta:
            return v
        alpha = max(alpha, v)
    return v

def min_value(board, alpha, beta, depth):
    if terminal(board) or depth == 0:
        return utility(board)

    v = math.inf
    for action in actions(board):
        v = min(v, max_value(result(board, action), alpha, beta, depth - 1))
        if v <= alpha:
            return v
        beta = min(beta, v)
    return v

# ===== 10. Hàm minimax chính =====
def minimax(board, depth=3):  # depth=3 đủ cho 5×5, tăng thành 2 cho 10×10
    current = player(board)
    best_move = None

    if current == X:  # X là người chơi tối đa hóa
        best_val = -math.inf
        for action in actions(board):
            value = min_value(result(board, action), -math.inf, math.inf, depth)
            if value > best_val:
                best_val = value
                best_move = action
    else:  # O là người chơi tối thiểu hóa
        best_val = math.inf
        for action in actions(board):
            value = max_value(result(board, action), -math.inf, math.inf, depth)
            if value < best_val:
                best_val = value
                best_move = action

    return best_move

def print_board_display(board):
    for row in board:
        print(" | ".join([cell if cell is not EMPTY else " " for cell in row]))
        if row != board[-1]:
            print("-" * (len(row) * 4 - 3))

def main():
    global WIN_LENGTH

    print("Chào mừng đến với trò chơi Tic-Tac-Toe/Gomoku!")

    # Nhập kích thước bàn cờ N
    while True:
        try:
            N_input = int(input("Nhập kích thước bàn cờ N (ví dụ: 3 cho Tic-Tac-Toe, 5 cho Gomoku): "))
            if N_input < 3:
                print("Kích thước bàn cờ phải ít nhất là 3.")
            else:
                break
        except ValueError:
            print("Đầu vào không hợp lệ. Vui lòng nhập một số nguyên.")

    # Nhập độ dài chiến thắng K
    while True:
        try:
            K_input = int(input(f"Nhập số quân liên tiếp K để thắng (ví dụ: 3 cho Tic-Tac-Toe, 5 cho Gomoku trên bàn 5x5 trở lên. Tối đa là {N_input}): "))
            if K_input < 3 or K_input > N_input:
                print(f"Độ dài chiến thắng K phải nằm trong khoảng từ 3 đến {N_input}.")
            else:
                WIN_LENGTH = K_input
                break
        except ValueError:
            print("Đầu vào không hợp lệ. Vui lòng nhập một số nguyên.")

    board_size = N_input
    board = initial_state(board_size)

    user_player = None
    ai_player = None

    print("Chọn người chơi (X hoặc O):")
    user_choice = input().upper()

    while user_choice not in [X, O]:
        print("Lựa chọn không hợp lệ. Vui lòng chọn 'X' hoặc 'O'.")
        user_choice = input().upper()

    user_player = user_choice
    ai_player = X if user_player == O else O

    print(f"Bạn là {user_player}. AI là {ai_player}.")
    print("Trò chơi bắt đầu!")

    game_active = True
    while game_active:
        current_player_turn = player(board)

        if terminal(board):
            game_winner = winner(board)
            print_board_display(board)
            if game_winner:
                print(f"Chúc mừng! Người chơi {game_winner} đã thắng!")
            else:
                print("Trò chơi kết thúc: HÒA.")
            game_active = False
            break

        if current_player_turn == user_player:
            print(f"\nLượt của bạn ({user_player}).")
            print("Bàn cờ hiện tại:")
            print_board_display(board)

            valid_move = False
            while not valid_move:
                try:
                    row = int(input(f"Nhập hàng (0 đến {board_size - 1}): "))
                    col = int(input(f"Nhập cột (0 đến {board_size - 1}): "))

                    if 0 <= row < board_size and 0 <= col < board_size and board[row][col] == EMPTY:
                        board = result(board, (row, col))
                        valid_move = True
                    else:
                        print("Nước đi không hợp lệ. Vui lòng chọn một ô trống trong giới hạn bàn cờ.")
                except ValueError:
                    print("Đầu vào không hợp lệ. Vui lòng nhập số nguyên.")
        else: # AI's turn
            print(f"\nLượt của AI ({ai_player}). AI đang suy nghĩ...")
            # Đối với các bàn cờ lớn hơn hoặc độ sâu cao hơn, minimax có thể mất nhiều thời gian.
            # Điều chỉnh depth tùy theo kích thước N và K. depth=2 hoặc 3 là hợp lý cho N=5.
            # Đối với N lớn, cần một hàm đánh giá (evaluation function) tốt hơn và độ sâu thấp hơn.
            search_depth = 2 # Có thể điều chỉnh
            if board_size <= 4: # Tăng độ sâu cho bàn cờ nhỏ hơn
                search_depth = 4
            elif board_size <= 6:
                search_depth = 3
            else:
                search_depth = 2 # Giới hạn độ sâu để AI không quá chậm trên bàn lớn

            ai_move = minimax(board, depth=search_depth)
            if ai_move:
                print(f"AI chọn: {ai_move}")
                board = result(board, ai_move)
            else:
                # Điều này chỉ xảy ra nếu không còn nước đi nào và trò chơi chưa kết thúc,
                # hoặc thuật toán minimax không tìm được nước đi (khó xảy ra với actions()).
                print("AI không thể tìm thấy nước đi hợp lệ.")
                game_active = False

# Khởi chạy trò chơi khi script được chạy trực tiếp
if __name__ == "__main__":
    main()


In [ ]:
import math
import copy
import numpy as np
import os # Import os for clearing console in PrintBoard

X = "X"
O = "O"
EMPTY = None
# WIN_LENGTH sẽ được thiết lập bằng đầu vào người dùng
WIN_LENGTH = None


# ===========================
# 1. Tạo bàn NxN
# ===========================
def initial_state(N):
    return [[EMPTY for _ in range(N)] for _ in range(N)]


# ===========================
# 2. Xác định người chơi tiếp theo
# ===========================
def player(board):
    count = sum(1 for row in board for cell in row if cell is not None)
    return X if count % 2 == 0 else O


# ===========================
# 3. Sinh các nước đi hợp lệ
# ===========================
def actions(board):
    return {(i, j) for i in range(len(board))
            for j in range(len(board)) if board[i][j] is EMPTY}


# ===========================
# 4. Trạng thái mới sau khi đi
# ===========================
def result(board, action):
    i, j = action
    new_board = copy.deepcopy(board)
    new_board[i][j] = player(board)
    return new_board


# ===========================
# 5. Hàm kiểm tra K quân liên tiếp
# ===========================
def check_k_in_a_row(board, symbol, k):
    N = len(board)

    if k > N:
        return False

    # ---- kiểm tra hàng ngang ----
    for r in range(N):
        for c in range(N - k + 1):
            if all(board[r][c+l] == symbol for l in range(k)):
                return True

    # ---- kiểm tra hàng dọc ----
    for r in range(N - k + 1):
        for c in range(N):
            if all(board[r+l][c] == symbol for l in range(k)):
                return True

    # ---- kiểm tra chéo chính ----
    for r in range(N - k + 1):
        for c in range(N - k + 1):
            if all(board[r+l][c+l] == symbol for l in range(k)):
                return True

    # ---- kiểm tra chéo phụ ----
    for r in range(N - k + 1):
        for c in range(k - 1, N):
            if all(board[r+l][c-l] == symbol for l in range(k)):
                return True

    return False


# ===========================
# 6. Kiểm tra người thắng
# ===========================
def winner(board):
    # Sử dụng biến WIN_LENGTH toàn cục
    if check_k_in_a_row(board, X, WIN_LENGTH):
        return X
    if check_k_in_a_row(board, O, WIN_LENGTH):
        return O
    return None


# ===========================
# 7. Trạng thái kết thúc
# ===========================
def terminal(board):
    if winner(board) is not None:
        return True

    if not actions(board): # Không còn nước đi nào (hòa)
        return True

    return False


# ===========================
# 8. Hàm utility cho minimax
# ===========================
def utility(board):
    w = winner(board)
    if w == X:
        return 1
    if w == O:
        return -1
    return 0


# ==========================================================
# 9. Thuật toán Minimax + CẮT TỈA ALPHA-BETA
# ==========================================================
def max_value(board, alpha, beta, depth):
    if terminal(board) or depth == 0:
        return utility(board)

    v = -math.inf

    for action in actions(board):
        v = max(v, min_value(result(board, action), alpha, beta, depth - 1))

        if v >= beta:
            return v   # Cắt tỉa

        alpha = max(alpha, v)

    return v


def min_value(board, alpha, beta, depth):
    if terminal(board) or depth == 0:
        return utility(board)

    v = math.inf

    for action in actions(board):
        v = min(v, max_value(result(board, action), alpha, beta, depth - 1))

        if v <= alpha:
            return v   # Cắt tỉa

        beta = min(beta, v)

    return v


# ===========================
# 10. Hàm Minimax chính
# ===========================
def alpha_beta_search(board, depth=3):
    current = player(board)
    best_move = None

    if current == X:
        best_val = -math.inf
        for action in actions(board):
            val = min_value(result(board, action), -math.inf, math.inf, depth)
            if val > best_val:
                best_val = val
                best_move = action

    else:  # current == O
        best_val = math.inf
        for action in actions(board):
            val = max_value(result(board, action), -math.inf, math.inf, depth)
            if val < best_val:
                best_val = val
                best_move = action

    return best_move

# ===========================
# 11. Hàm hiển thị bàn cờ
# ===========================
def print_board_display(board):
    # os.system('cls' if os.name=='nt' else 'clear') # Uncomment to clear console on each move
    N = len(board)
    for r_idx, row in enumerate(board):
        display_row = []
        for c_idx, cell in enumerate(row):
            display_row.append(cell if cell is not EMPTY else f"({r_idx},{c_idx})") # Show coordinates for empty cells
        print(" | ".join(display_row))
        if r_idx != N - 1:
            print("-" * (len(display_row) * 6 - 3)) # Adjusted length for coordinate display

# ===========================
# 12. Vòng lặp chính của trò chơi (main function)
# ===========================
def main():
    global WIN_LENGTH

    print("Chào mừng đến với trò chơi Gomoku!")

    # Nhập kích thước bàn cờ N
    while True:
        try:
            N_input = int(input("Nhập kích thước bàn cờ N (ví dụ: 3 cho Tic-Tac-Toe, 5 cho Gomoku): "))
            if N_input < 3:
                print("Kích thước bàn cờ phải ít nhất là 3.")
            else:
                break
        except ValueError:
            print("Đầu vào không hợp lệ. Vui lòng nhập một số nguyên.")

    # Nhập độ dài chiến thắng K
    while True:
        try:
            K_input = int(input(f"Nhập số quân liên tiếp K để thắng (ví dụ: 3 cho Tic-Tac-Toe, 5 cho Gomoku trên bàn 5x5 trở lên. Tối đa là {N_input}): "))
            if K_input < 3 or K_input > N_input:
                print(f"Độ dài chiến thắng K phải nằm trong khoảng từ 3 đến {N_input}.")
            else:
                WIN_LENGTH = K_input
                break
        except ValueError:
            print("Đầu vào không hợp lệ. Vui lòng nhập một số nguyên.")

    board_size = N_input
    board = initial_state(board_size)

    user_player = None
    ai_player = None

    print("Chọn người chơi (X hoặc O):")
    user_choice = input().upper()

    while user_choice not in [X, O]:
        print("Lựa chọn không hợp lệ. Vui lòng chọn 'X' hoặc 'O'.")
        user_choice = input().upper()

    user_player = user_choice
    ai_player = X if user_player == O else O

    print(f"Bạn là {user_player}. AI là {ai_player}.")
    print(f"Trò chơi bắt đầu với bàn cờ {board_size}x{board_size}. Ai cần {WIN_LENGTH} quân liên tiếp để thắng.")

    game_active = True
    while game_active:
        current_player_turn = player(board)

        if terminal(board):
            game_winner = winner(board)
            print_board_display(board)
            if game_winner:
                print(f"Chúc mừng! Người chơi {game_winner} đã thắng!")
            else:
                print("Trò chơi kết thúc: HÒA.")
            game_active = False
            break

        if current_player_turn == user_player:
            print(f"\nLượt của bạn ({user_player}).")
            print("Bàn cờ hiện tại:")
            print_board_display(board)

            valid_move = False
            while not valid_move:
                try:
                    row = int(input(f"Nhập hàng (0 đến {board_size - 1}): "))
                    col = int(input(f"Nhập cột (0 đến {board_size - 1}): "))

                    if 0 <= row < board_size and 0 <= col < board_size and board[row][col] == EMPTY:
                        board = result(board, (row, col))
                        valid_move = True
                    else:
                        print("Nước đi không hợp lệ. Vui lòng chọn một ô trống trong giới hạn bàn cờ.")
                except ValueError:
                    print("Đầu vào không hợp lệ. Vui lòng nhập số nguyên.")
        else: # AI's turn
            print(f"\nLượt của AI ({ai_player}). AI đang suy nghĩ...")
            # Điều chỉnh độ sâu tìm kiếm cho phù hợp với kích thước bàn cờ
            # Đối với bàn cờ 10x10, độ sâu 2 hoặc 3 là hợp lý để không quá chậm.
            search_depth = 2
            if board_size <= 4: # Tăng độ sâu cho bàn cờ nhỏ hơn
                search_depth = 4
            elif board_size <= 6:
                search_depth = 3
            else:
                search_depth = 2 # Giới hạn độ sâu để AI không quá chậm trên bàn lớn

            ai_move = alpha_beta_search(board, depth=search_depth)
            if ai_move:
                print(f"AI chọn: {ai_move}")
                board = result(board, ai_move)
            else:
                print("AI không thể tìm thấy nước đi hợp lệ. Trò chơi kết thúc.")
                game_active = False


# Khởi chạy trò chơi khi script được chạy trực tiếp
if __name__ == "__main__":
    main()

In [9]:
# Chess Alpha-Beta example (run in Colab)
!pip install python-chess >/dev/null

import chess
import chess.polyglot
import math
import time

# Simple piece values
PIECE_VALUES = {
    chess.PAWN: 100,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 20000
}

# Simple evaluation: material + small mobility bonus
def evaluate(board: chess.Board):
    if board.is_checkmate():
        # If side to move is checkmated -> very bad
        return -999999 if board.turn else 999999
    if board.is_stalemate() or board.is_insufficient_material():
        return 0

    value = 0
    for piece_type in PIECE_VALUES:
        value += len(board.pieces(piece_type, chess.WHITE)) * PIECE_VALUES[piece_type]
        value -= len(board.pieces(piece_type, chess.BLACK)) * PIECE_VALUES[piece_type]

    # mobility: number of legal moves difference (small weight)
    w_mob = len(list(board.legal_moves))
    # flip board to get opponent's mobility (simulate turn)
    board.push(chess.Move.null())
    b_mob = len(list(board.legal_moves))
    board.pop()
    value += 10 * (w_mob - b_mob)

    return value if board.turn else -value  # perspective: positive is good for side to move

# Alpha-beta with basic move ordering (captures first)
def order_moves(board, moves):
    # simple: prefer captures and promotions
    scored = []
    for m in moves:
        score = 0
        if board.is_capture(m):
            # MVV/LVA simple: bigger captured piece => higher score
            captured = board.piece_type_at(m.to_square)
            if captured:
                score += PIECE_VALUES.get(captured, 0)
        if m.promotion:
            score += PIECE_VALUES[chess.QUEEN]
        scored.append((score, m))
    scored.sort(key=lambda x: -x[0])
    return [m for _, m in scored]

def alpha_beta(board, depth, alpha, beta, start_time, time_limit=5.0):
    # terminal or depth
    if depth == 0 or board.is_game_over() or (time.time() - start_time) > time_limit:
        return evaluate(board), None

    best_move = None
    if board.turn:  # White maximizes
        max_eval = -math.inf
        moves = order_moves(board, list(board.legal_moves))
        for m in moves:
            board.push(m)
            eval_child, _ = alpha_beta(board, depth-1, alpha, beta, start_time, time_limit)
            board.pop()
            if eval_child > max_eval:
                max_eval = eval_child
                best_move = m
            alpha = max(alpha, eval_child)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:  # Black minimizes
        min_eval = math.inf
        moves = order_moves(board, list(board.legal_moves))
        for m in moves:
            board.push(m)
            eval_child, _ = alpha_beta(board, depth-1, alpha, beta, start_time, time_limit)
            board.pop()
            if eval_child < min_eval:
                min_eval = eval_child
                best_move = m
            beta = min(beta, eval_child)
            if beta <= alpha:
                break
        return min_eval, best_move

# Example usage:
board = chess.Board()  # starting position
print(board)

# iterative deepening for a limited time
time_limit = 3.0  # seconds
start = time.time()
best = None
for d in range(1, 5):  # up to 4-ply by default
    val, mv = alpha_beta(board, d, -math.inf, math.inf, start, time_limit)
    if (time.time() - start) > time_limit:
        break
    if mv:
        best = mv
    print(f"Depth {d} -> eval {val}, move {mv}")
print("Best chosen move:", best)


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
Depth 1 -> eval 100, move e2e3
Depth 2 -> eval 0, move e2e3
Depth 3 -> eval 150, move e2e3
Depth 4 -> eval 0, move e2e3
Best chosen move: e2e3
